In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
import csv

# Set the root directory (Main_Dataset_Folder) containing the master folders
root_dir = r"/kaggle/input/indic-leaf-ds/Indic-Leaf-Dataset-master/Indic-Leaf-Dataset-master/indic_leaf_dataset"  # change this to your main dataset folder path
output_csv = "indicleaf123.csv"

def extract_class_label(folder_name):
    """
    Extracts the class label by splitting the folder name on the first underscore.
    E.g., "001_Rose" returns "Rose". If no underscore exists, returns the folder name.
    """
    if "_" in folder_name:
        return folder_name.split("_", 1)[1]
    return folder_name

# Open CSV file for writing
with open(output_csv, mode="w", newline="") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["filepath", "class_label"])  # CSV header

    # Loop over each master folder in the root directory
    for master_folder_name in os.listdir(root_dir):
        master_folder = os.path.join(root_dir, master_folder_name)
        if os.path.isdir(master_folder):
            # Extract class label from master folder name (remove number_ prefix)
            class_label = extract_class_label(master_folder_name)
            # Walk through all subdirectories under the master folder
            for subdir, _, files in os.walk(master_folder):
                for file in files:
                    # Check if the file is an image (adjust extensions if needed)
                    if file.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".gif")):
                        file_path = os.path.join(subdir, file)
                        writer.writerow([file_path, class_label])

print(f"CSV file '{output_csv}' has been created.")

CSV file 'indicleaf123.csv' has been created.


In [3]:
from PIL import Image, UnidentifiedImageError
import pandas as pd

# Load CSV
df = pd.read_csv('/kaggle/input/indiccsv/indicleaf123.csv')

# Define a function to check image validity
def is_valid_image(path):
    try:
        with Image.open(path) as img:
            img.verify()  # Check image integrity
        return True
    except (UnidentifiedImageError, Exception) as e:
        print(f"Removing {path}: {e}")
        return False

# Apply the function to each row's filepath.
# This returns a boolean Series.
mask = df['filepath'].apply(is_valid_image)

# Filter the DataFrame for valid images
df_clean = df[mask].reset_index(drop=True)

# Optionally, save the cleaned CSV
df_clean.to_csv('cleaned_indicleaf.csv', index=False)
print(f"Cleaned CSV saved with {len(df_clean)} rows (out of {len(df)})")


Removing /kaggle/input/indic-leaf-ds/Indic-Leaf-Dataset-master/Indic-Leaf-Dataset-master/indic_leaf_dataset/99_ti_plant/level_3/IMG_20180825_155235.jpg: cannot identify image file '/kaggle/input/indic-leaf-ds/Indic-Leaf-Dataset-master/Indic-Leaf-Dataset-master/indic_leaf_dataset/99_ti_plant/level_3/IMG_20180825_155235.jpg'
Removing /kaggle/input/indic-leaf-ds/Indic-Leaf-Dataset-master/Indic-Leaf-Dataset-master/indic_leaf_dataset/49_spider_lily/level_1/IMG_20180821_170400.jpg: cannot identify image file '/kaggle/input/indic-leaf-ds/Indic-Leaf-Dataset-master/Indic-Leaf-Dataset-master/indic_leaf_dataset/49_spider_lily/level_1/IMG_20180821_170400.jpg'
Removing /kaggle/input/indic-leaf-ds/Indic-Leaf-Dataset-master/Indic-Leaf-Dataset-master/indic_leaf_dataset/95_royal_palm/level_3/IMG_20180518_171351.jpg: cannot identify image file '/kaggle/input/indic-leaf-ds/Indic-Leaf-Dataset-master/Indic-Leaf-Dataset-master/indic_leaf_dataset/95_royal_palm/level_3/IMG_20180518_171351.jpg'
Cleaned CSV sav

In [1]:
#fine tune
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.applications import ConvNeXtBase
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping , ReduceLROnPlateau
from tensorflow.keras.applications.convnext import preprocess_input
from PIL import ImageFile
from sklearn.metrics import classification_report
import numpy as np

# Enable loading of truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Load and prepare the CSV data
df = pd.read_csv('/kaggle/input/indiccsv/cleaned_indicleaf.csv')
df['filepath'] = df['filepath'].astype(str)
df['class_label'] = df['class_label'].astype(str)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Split data into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['class_label'])

# Create data generators with ConvNeXt preprocessing
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

# Generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filepath',
    y_col='class_label',
    target_size=(224, 224),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='filepath',
    y_col='class_label',
    target_size=(224, 224),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Get number of classes
num_classes = len(train_generator.class_indices)

# Build ConvNeXtBase model
base_model = ConvNeXtBase(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

# Freeze the base model
base_model.trainable = False

# Add custom head
inputs = Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs, outputs)

# Compile with higher learning rate
model.compile(
    optimizer=Adam(learning_rate=0.0001),  
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
checkpoint = ModelCheckpoint(
    'best_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Train the model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[checkpoint, early_stop]
)

# Fine-tuning phase: unfreeze top layers
base_model.trainable = True
for layer in base_model.layers[:-80]:  # Unfreeze last 80 layers
    layer.trainable = False

# Recompile with lower learning rate
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='sparscategorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tuning callbacks
fine_tune_callbacks = [
    ModelCheckpoint(
        'fine_tuned_model.keras',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max'
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5,
        min_lr=1e-7
    )
]

# Fine-tuning training
history_fine = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=fine_tune_callbacks
)

# Load best weights and save final model
model.load_weights('best_model.keras')
model.save('plant_classification_indic_convnext.keras')

# Evaluation
results = model.evaluate(val_generator)
print(f'Validation Loss: {results[0]}, Validation Accuracy: {results[1]}')

val_preds = model.predict(val_generator, verbose=1)
val_preds_class = np.argmax(val_preds, axis=1)

true_labels = val_generator.classes
label_map = {v: k for k, v in train_generator.class_indices.items()}
target_names = [label_map[i] for i in range(num_classes)]

report = classification_report(true_labels, val_preds_class, target_names=target_names)
print(report)

Found 22384 validated image filenames belonging to 112 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 20 invalid image filename(s) in x_col="filepath". These filename(s) will be ignored.
  warnings.warn(


Found 5595 validated image filenames belonging to 112 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 6 invalid image filename(s) in x_col="filepath". These filename(s) will be ignored.
  warnings.warn(


350926856/350926856 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


700/700 ━━━━━━━━━━━━━━━━━━━━ 735s 1s/step - accuracy: 0.1560 - loss: 4.1071 - val_accuracy: 0.6055 - val_loss: 2.0164
Epoch 2/20
700/700 ━━━━━━━━━━━━━━━━━━━━ 477s 675ms/step - accuracy: 0.5930 - loss: 1.9128 - val_accuracy: 0.7369 - val_loss: 1.2465
Epoch 3/20
700/700 ━━━━━━━━━━━━━━━━━━━━ 470s 665ms/step - accuracy: 0.7036 - loss: 1.2980 - val_accuracy: 0.7786 - val_loss: 0.9863
Epoch 4/20
700/700 ━━━━━━━━━━━━━━━━━━━━ 470s 665ms/step - accuracy: 0.7632 - loss: 1.0139 - val_accuracy: 0.8077 - val_loss: 0.8367
Epoch 5/20
700/700 ━━━━━━━━━━━━━━━━━━━━ 467s 661ms/step - accuracy: 0.7923 - loss: 0.8655 - val_accuracy: 0.8211 - val_loss: 0.7448
Epoch 6/20
700/700 ━━━━━━━━━━━━━━━━━━━━ 465s 659ms/step - accuracy: 0.8193 - loss: 0.7397 - val_accuracy: 0.8340 - val_loss: 0.6793
Epoch 7/20
700/700 ━━━━━━━━━━━━━━━━━━━━ 465s 659ms/step - accuracy: 0.8340 - loss: 0.6817 - val_accuracy: 0.8445 - val_loss: 0.6308
Epoch 8/20
700/700 ━━━━━━━━━━━━━━━━━━━━ 467s 661ms/step - accuracy: 0.8555 - loss: 0.6112 

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 220 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


175/175 ━━━━━━━━━━━━━━━━━━━━ 76s 431ms/step - accuracy: 0.8926 - loss: 0.4103
Validation Loss: 0.40541499853134155, Validation Accuracy: 0.8947274088859558
175/175 ━━━━━━━━━━━━━━━━━━━━ 83s 448ms/step
                   precision    recall  f1-score   support

          acalypa       0.98      0.91      0.95        67
african_milk_tree       1.00      0.85      0.92        13
        air_plant       0.94      0.94      0.94        54
   allamanda_pink       0.95      0.86      0.90        49
             amla       0.92      0.84      0.88        56
       arrow_head       0.89      0.92      0.91        63
     ashokachettu       0.90      0.93      0.92        71
            badam       0.87      0.91      0.89        64
           bamboo       0.98      0.96      0.97        45
           banana       1.00      0.94      0.97        36
           banyan       0.79      0.84      0.82        58
       blackberry       0.91      0.82      0.87        51
          boxwood       0.86    

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import joblib

# Assume your CSV has a column 'class_label'
df = pd.read_csv('/kaggle/input/indiccsv/cleaned_indicleaf.csv')

# Initialize and fit the encoder on your class labels
encoder = LabelEncoder()
df['encoded_label'] = encoder.fit_transform(df['class_label'])

# Optionally, you can save the encoder for later use in prediction
joblib.dump(encoder, 'label_encoder.pkl')

# Continue with your training process using the 'encoded_label' column or your generator.


In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import joblib

def predict_image(image_path, model_path, encoder_path=None):
    # Load the trained model (.keras format)
    model = load_model(model_path)
    
    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(224, 224), color_mode='rgb')
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Predict the class probabilities
    predictions = model.predict(img_array)
    
    # Get the index of the most dominant class
    dominant_index = np.argmax(predictions, axis=1)[0]
    
    # If an encoder path is provided, load the encoder and convert the index to the actual label
    if encoder_path is not None:
        encoder = joblib.load(encoder_path)
        dominant_label = encoder.inverse_transform([dominant_index])[0]
    else:
        dominant_label = dominant_index  # Fallback to numeric label if encoder is not provided
    
    return dominant_label, predictions

# Example usage
if __name__ == '__main__':
    image_path = '/kaggle/input/img3333/6b7284dfeab7222691aa7e7472d8d4bc.jpg'
    model_path = '/kaggle/working/plant_classification_indic_convnext.keras'
    encoder_path = '/kaggle/working/label_encoder.pkl'  # Path to your saved encoder
    
    dominant_label, preds = predict_image(image_path, model_path, encoder_path)
    print("Predicted Class:", dominant_label)


In [ ]:
df = pd.read_csv('/kaggle/input/indiccsv/cleaned_indicleaf.csv')

In [ ]:
df['class_label'].unique()